# DS 4002 Project 3

## Data Cleaning

In [ ]:
#Standard imports
import pandas as pd
import numpy as np

In [ ]:
#Install and import package for continents
!pip install pycountry_convert
import pycountry_convert as pc

In [ ]:
#Read in data
WHR = pd.read_excel(r"/content/WHR Data.xls")

In [ ]:
#Original number of countries
WHR["Country name"].nunique()

165

In [ ]:
#Drop rows with nan values
WHR = WHR.dropna()
#Drop irrelevant columns
WHR = WHR.drop(columns=["Positive affect","Negative affect"])
#Rename columns to more user friendly names
WHR = WHR.rename(columns={"Country name":"Country","year":"Year","Life Ladder":"Happiness Score","Log GDP per capita":"Log GDP per Cap","Social support":"Social Support","Healthy life expectancy at birth":"Life Expectancy",
                    "Freedom to make life choices":"Freedom","Perceptions of corruption":"Corruption"})

In [ ]:
#Select only countries with complete data from 2013 to 2019
WHR = WHR[WHR["Year"].isin(range(2013,2020))]
countries = np.unique(WHR["Country"], return_counts=True)[0]
occurances = np.unique(WHR["Country"], return_counts=True)[1] == 7
dropCountries = countries[np.where(occurances)]
WHR = WHR[WHR["Country"].isin(dropCountries)]

In [ ]:
WHR

,Country,Year,Happiness Score,Log GDP per Cap,Social Support,Life Expectancy,Freedom,Generosity,Corruption
5,Afghanistan,2013,3.572100,7.680333,0.483552,52.000000,0.577955,0.062666,0.823204
6,Afghanistan,2014,3.130896,7.670638,0.525568,52.299999,0.508514,0.105755,0.871242
7,Afghanistan,2015,3.982855,7.653833,0.528597,52.599998,0.388928,0.081652,0.880638
8,Afghanistan,2016,4.220169,7.650370,0.559072,52.924999,0.522566,0.043916,0.793246
9,Afghanistan,2017,2.661718,7.647830,0.490880,53.250000,0.427011,-0.119410,0.954393
...,...,...,...,...,...,...,...,...,...
2191,Zimbabwe,2015,3.703191,7.746680,0.735800,51.200001,0.667193,-0.107365,0.810457
2192,Zimbabwe,2016,3.735400,7.734831,0.768425,51.674999,0.732971,-0.078541,0.723612
2193,Zimbabwe,2017,3.638300,7.754387,0.754147,52.150002,0.752826,-0.080725,0.751208
2194,Zimbabwe,2018,3.616480,7.783066,0.775388,52.625000,0.762675,-0.051219,0.844209


In [ ]:
#Rename countries with different names to alpha-2 country code list
WHR["Country"] = WHR["Country"].replace(["Congo (Brazzaville)","Turkiye"],["Congo","Turkey"])

In [ ]:
#New number of countries
WHR["Country"].nunique()

103

In [ ]:
#Set dataframe index to Country so it's easier to drop countries
WHR = WHR.set_index("Country")

In [ ]:
#Assign continents
WHR["Country Code"] = WHR.index.map(lambda x: pc.country_name_to_country_alpha2(x, cn_name_format="default"))
WHR["Continent Code"] = WHR["Country Code"].map(lambda x: pc.country_alpha2_to_continent_code(x))
WHR["Continent"] = WHR["Continent Code"].map(lambda x: pc.convert_continent_code_to_continent_name(x))
WHR = WHR.drop(columns=["Country Code", "Continent Code"]).reset_index()

In [ ]:
#Cleaned dataset
WHR

,Country,Year,Happiness Score,Log GDP per Cap,Social Support,Life Expectancy,Freedom,Generosity,Corruption,Continent
0,Afghanistan,2013,3.572100,7.680333,0.483552,52.000000,0.577955,0.062666,0.823204,Asia
1,Afghanistan,2014,3.130896,7.670638,0.525568,52.299999,0.508514,0.105755,0.871242,Asia
2,Afghanistan,2015,3.982855,7.653833,0.528597,52.599998,0.388928,0.081652,0.880638,Asia
3,Afghanistan,2016,4.220169,7.650370,0.559072,52.924999,0.522566,0.043916,0.793246,Asia
4,Afghanistan,2017,2.661718,7.647830,0.490880,53.250000,0.427011,-0.119410,0.954393,Asia
...,...,...,...,...,...,...,...,...,...,...
716,Zimbabwe,2015,3.703191,7.746680,0.735800,51.200001,0.667193,-0.107365,0.810457,Africa
717,Zimbabwe,2016,3.735400,7.734831,0.768425,51.674999,0.732971,-0.078541,0.723612,Africa
718,Zimbabwe,2017,3.638300,7.754387,0.754147,52.150002,0.752826,-0.080725,0.751208,Africa
719,Zimbabwe,2018,3.616480,7.783066,0.775388,52.625000,0.762675,-0.051219,0.844209,Africa


In [ ]:
#Exporting dataset
WHR.to_csv('WHR.csv', index=False)

## Data Vizualization

In [ ]:
#Data viz import
import plotly.express as px

In [ ]:
#Happiness Score by Continent over Time
px.line(WHR.groupby(by=["Continent","Year"]).agg("mean").reset_index(),"Year","Happiness Score",color="Continent")

<ipython-input-5-e17dd27721e0>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  px.line(WHR.groupby(by=["Continent","Year"]).agg("mean").reset_index(),"Year","Happiness Score",color="Continent")


In [ ]:
#Happiness Score vs GDP
px.scatter(WHR, "Log GDP per Cap", "Happiness Score", color = "Continent", hover_name = "Country", animation_frame="Year", animation_group="Country", title="Happiness Score vs. GDP")

In [ ]:
#Happiness Score vs Social Support
px.scatter(WHR, "Social Support", "Happiness Score", color = "Continent", hover_name = "Country", animation_frame="Year", animation_group="Country", title="Happiness Score vs. Social Support")

In [ ]:
#Happiness Score vs Life Expectancy
px.scatter(WHR, "Life Expectancy", "Happiness Score", color = "Continent", hover_name = "Country", animation_frame="Year", animation_group="Country", title="Happiness Score vs. Life Expectancy")

In [ ]:
#Happiness Score vs Freedom
px.scatter(WHR, "Freedom", "Happiness Score", color = "Continent", hover_name = "Country", animation_frame="Year", animation_group="Country", title="Happiness Score vs. Freedom")

In [ ]:
#Happiness Score vs Generosity
px.scatter(WHR, "Generosity", "Happiness Score", color = "Continent", hover_name = "Country", animation_frame="Year", animation_group="Country", title="Happiness Score vs. Generosity")

In [ ]:
#Happiness Score vs Corruption
px.scatter(WHR, "Corruption", "Happiness Score", color = "Continent", hover_name = "Country", animation_frame="Year", animation_group="Country", title="Happiness Score vs. Corruption")

## Machine Learning

In [ ]:
WHR = pd.read_csv(r"/content/WHR.csv")

In [ ]:
WHR.head()

,Country,Year,Happiness Score,Log GDP per Cap,Social Support,Life Expectancy,Freedom,Generosity,Corruption,Continent
0,Afghanistan,2013,3.572100,7.680333,0.483552,52.000000,0.577955,0.062666,0.823204,Asia
1,Afghanistan,2014,3.130896,7.670638,0.525568,52.299999,0.508514,0.105755,0.871242,Asia
2,Afghanistan,2015,3.982855,7.653833,0.528597,52.599998,0.388928,0.081652,0.880638,Asia
3,Afghanistan,2016,4.220169,7.650370,0.559072,52.924999,0.522566,0.043916,0.793246,Asia
4,Afghanistan,2017,2.661718,7.647830,0.490880,53.250000,0.427011,-0.119410,0.954393,Asia


In [ ]:
#Machine learning imports
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [ ]:
#Keep columns relavent for ML
ML_WHR = WHR.drop(columns=["Country", "Year", "Continent", "Generosity", "Corruption"])

#Divide into X & y
X = StandardScaler().fit_transform(ML_WHR.drop(columns=["Happiness Score"]))
y = ML_WHR["Happiness Score"]
#Split data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
#Create and train model
mult_reg = LinearRegression()
mult_reg.fit(X_train,y_train)

LinearRegression()

In [ ]:
#Generate Predictions
predicted = mult_reg.predict(X_test)
actual = np.array(y_test)

#MSE & RMSE
mse = mean_squared_error(predicted,actual)
rmse = mean_squared_error(predicted,actual,squared=False)
print("MSE is ", round(mse,5), " and RMSE is ", round(rmse,5))

MSE is  0.35657  and RMSE is  0.59714


In [ ]:
#Beta Coefficients
coef = np.around(mult_reg.coef_,3)
intercept = round(mult_reg.intercept_,3)
factor_impact = (coef - coef[3]) / coef[3]

In [ ]:
print(factor_impact)

[0.01574803 0.12598425 0.51968504 0.        ]


In [ ]:
#Multiple Linear Regression Equation
print("Score =", coef[0],"*GDPperCap +", coef[1],"*SocialSup +",coef[2],"*LifeExpect +",coef[3],"*Freedom +", intercept)

Score = 0.258 *GDPperCap + 0.286 *SocialSup + 0.386 *LifeExpect + 0.254 *Freedom + 5.507


## Future Analysis

In [ ]:
WHR22 = pd.read_excel(r"/content/WHR Data.xls")

In [ ]:
#Drop rows with nan values
WHR22 = WHR22.dropna()
#Drop irrelevant columns
WHR22 = WHR22.drop(columns=["Positive affect","Negative affect"])
#Rename columns to more user friendly names
WHR22 = WHR22.rename(columns={"Country name":"Country","year":"Year","Life Ladder":"Happiness Score","Log GDP per capita":"Log GDP per Cap","Social support":"Social Support","Healthy life expectancy at birth":"Life Expectancy",
                    "Freedom to make life choices":"Freedom","Perceptions of corruption":"Corruption"})

In [ ]:
#Select only 2022 data
WHR22 = WHR22[WHR22["Year"] == 2022].reset_index(drop=True)

In [ ]:
#Scale factors just as they were done for ML
WHR22[["Log GDP per Cap","Social Support","Life Expectancy","Freedom"]] = StandardScaler().fit_transform(WHR22[["Log GDP per Cap","Social Support","Life Expectancy","Freedom"]])

In [ ]:
WHR22

,Country,Year,Happiness Score,Log GDP per Cap,Social Support,Life Expectancy,Freedom,Generosity,Corruption
0,Albania,2022,5.212213,0.108678,-0.520336,0.685061,0.069304,-0.065987,0.845502
1,Argentina,2022,6.260993,0.431835,0.670467,0.346901,0.272688,-0.127506,0.810037
2,Armenia,2022,5.381943,0.143337,0.092372,0.465477,-0.042865,-0.154325,0.704730
3,Australia,2022,7.034696,1.138830,1.010622,1.027611,0.526156,0.153465,0.545217
4,Austria,2022,6.998997,1.210516,0.550555,1.053962,0.541455,0.139042,0.524212
...,...,...,...,...,...,...,...,...,...
99,United States,2022,6.692790,1.327782,0.719243,0.079008,-0.521275,0.190581,0.701128
100,Uruguay,2022,6.670853,0.492883,0.751352,0.390817,0.740644,-0.051668,0.631337
101,Uzbekistan,2022,6.016239,-0.425785,0.569102,0.057049,1.459256,0.308951,0.615844
102,Vietnam,2022,6.266509,-0.137834,0.567840,0.057049,1.604537,-0.178987,0.703423


In [ ]:
#Create new column with predictions
predictions = []
for i in WHR22.index:
   predictions = np.append(predictions, coef[0]*WHR22["Log GDP per Cap"][i] + coef[1]*WHR22["Social Support"][i] + coef[2]*WHR22["Life Expectancy"][i] +
                           coef[3]*WHR22["Freedom"][i] + intercept)
WHR22["Predicted Happiness Score"] = predictions

In [ ]:
WHR22

,Country,Year,Happiness Score,Log GDP per Cap,Social Support,Life Expectancy,Freedom,Generosity,Corruption,Predicted Happiness Score
0,Albania,2022,5.212213,0.108678,-0.520336,0.685061,0.069304,-0.065987,0.845502,5.668260
1,Argentina,2022,6.260993,0.431835,0.670467,0.346901,0.272688,-0.127506,0.810037,6.013333
2,Armenia,2022,5.381943,0.143337,0.092372,0.465477,-0.042865,-0.154325,0.704730,5.739186
3,Australia,2022,7.034696,1.138830,1.010622,1.027611,0.526156,0.153465,0.545217,6.620158
4,Austria,2022,6.998997,1.210516,0.550555,1.053962,0.541455,0.139042,0.524212,6.521131
...,...,...,...,...,...,...,...,...,...,...
99,United States,2022,6.692790,1.327782,0.719243,0.079008,-0.521275,0.190581,0.701128,5.953365
100,Uruguay,2022,6.670853,0.492883,0.751352,0.390817,0.740644,-0.051668,0.631337,6.188030
101,Uzbekistan,2022,6.016239,-0.425785,0.569102,0.057049,1.459256,0.308951,0.615844,5.952583
102,Vietnam,2022,6.266509,-0.137834,0.567840,0.057049,1.604537,-0.178987,0.703423,6.063415


In [ ]:
#MSE & RMSE
mse = mean_squared_error(predictions,WHR22["Happiness Score"])
rmse = mean_squared_error(predictions,WHR22["Happiness Score"],squared=False)
print("MSE is ", round(mse,5), " and RMSE is ", round(rmse,5))

MSE is  0.35691  and RMSE is  0.59742
